In [1]:
import pickle
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.externals import joblib
import numpy as np
from pandas import DataFrame
from IPython.display import display

In [2]:
def display_metrics(true_labels, predicted_labels, labels):
    print("Accuracy:", accuracy_score(true_labels, predicted_labels))
    print("\nConfusion matrix")
    display(DataFrame(confusion_matrix(true_labels, predicted_labels),
                      index=["Actual " + label for label in labels], columns=["Predicted " + label for label in labels]))
    print("\nPrecision, recall, F-score, support")
    display(DataFrame(np.array(precision_recall_fscore_support(true_labels, predicted_labels)),
            index=("precision", "recall", "F-score", "support"), columns=labels))

# SVM Classifier with features extracted from the last layer of ResNet50

The general idea here is to use transfer learning approach. Inputs to the model are image representations in the form of vectors of length 2048, which are obtained after propagating images through a ResNet50 model pretrained on ImageNet and taking the outputs of its last layer before classification.

Having gotten the features, we need to classify the images into two classes: "good" cars and "damaged" cars. Here we'll try to use an SVM classifier, as it is suitable for binary classification of objects represented as feature vectors.

In [3]:
with open("train_resnet50.p", "rb") as train_pickle:
    train_cars, train_labels = pickle.load(train_pickle)

In [4]:
print(train_cars[0], train_cars[0].shape)
print(train_labels[0])

[0.         0.         0.04767787 ... 2.9299936  0.         0.        ] (2048,)
1


Let's begin with SVM classifier with a default RBF kernel.

In [5]:
clf_rbf = svm.SVC()
clf_rbf.fit(train_cars, train_labels) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [6]:
predicted_labels = clf_rbf.predict(train_cars)

In [7]:
display_metrics(train_labels, predicted_labels, ("Good Cars", "Damaged Cars"))

Accuracy: 0.8452563708934603

Confusion matrix


,Predicted Good Cars,Predicted Damaged Cars
Actual Good Cars,2469,729
Actual Damaged Cars,279,3037



Precision, recall, F-score, support


,Good Cars,Damaged Cars
precision,0.898472,0.806426
recall,0.772045,0.915862
F-score,0.830474,0.857667
support,3198.000000,3316.000000


The results are a worse than for an SVM classifier with an RBF kernel with features extracted from VGG-19. Let's see the results for the validation set.

In [8]:
with open("valid_resnet50.p", "rb") as valid_pickle:
    valid_cars, valid_labels = pickle.load(valid_pickle)

In [9]:
predicted_labels_valid = clf_rbf.predict(valid_cars)

In [10]:
display_metrics(valid_labels, predicted_labels_valid, ("Good Cars", "Damaged Cars"))

Accuracy: 0.855036855036855

Confusion matrix


,Predicted Good Cars,Predicted Damaged Cars
Actual Good Cars,310,83
Actual Damaged Cars,35,386



Precision, recall, F-score, support


,Good Cars,Damaged Cars
precision,0.898551,0.823028
recall,0.788804,0.916865
F-score,0.840108,0.867416
support,393.000000,421.000000


At least the results seem consistent.

For the sake of comparison let's also train a linear classifier.

In [11]:
clf_lin = svm.SVC(kernel='linear')
clf_lin.fit(train_cars, train_labels) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [12]:
predicted_labels = clf_lin.predict(train_cars)

In [13]:
display_metrics(train_labels, predicted_labels, ("Good Cars", "Damaged Cars"))

Accuracy: 0.9329137242861529

Confusion matrix


,Predicted Good Cars,Predicted Damaged Cars
Actual Good Cars,2947,251
Actual Damaged Cars,186,3130



Precision, recall, F-score, support


,Good Cars,Damaged Cars
precision,0.940632,0.925762
recall,0.921513,0.943908
F-score,0.930975,0.934747
support,3198.000000,3316.000000


In [14]:
predicted_labels_valid = clf_lin.predict(valid_cars)

In [15]:
display_metrics(valid_labels, predicted_labels_valid, ("Good Cars", "Damaged Cars"))

Accuracy: 0.9004914004914005

Confusion matrix


,Predicted Good Cars,Predicted Damaged Cars
Actual Good Cars,347,46
Actual Damaged Cars,35,386



Precision, recall, F-score, support


,Good Cars,Damaged Cars
precision,0.908377,0.893519
recall,0.882952,0.916865
F-score,0.895484,0.905041
support,393.000000,421.000000


The linear classifier again performs better as in the case of the VGG-19 features. But the overall performance is still worse than for linear SVM in that model.

Let's save the trained linear SVM classifier.

In [16]:
joblib.dump(clf_lin, 'svm_resnet50.pkl')

['svm_resnet50.pkl']